## ovr-svm

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from sklearn.metrics import f1_score, precision_score, recall_score,average_precision_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression,SGDClassifier

from joblib import Parallel, delayed

from tqdm import *

%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../../')
sys.path.append(src_dir)

In [14]:
%aimport src.data.movielens_20m_imdb
%aimport src.helpers.labels,src.helpers.neighbours, src.helpers.segments,src.helpers.embeddings
%aimport src.utils.dataframes, src.utils.clusters, src.utils.metrics,src.utils.plotting

In [18]:
from src.data.movielens_20m_imdb import load_df_or_get_from_cache
from src.helpers.labels import truncate_labels
from src.helpers.neighbours import get_predicted_labels_from_neighbours
from src.helpers.segments import make_distance_matrix_for_segments,vectorize_segments
from src.helpers.embeddings import read_glove_wiki_weighted


from src.utils.dataframes import sample_rows
from src.utils.metrics import ranking
from src.utils.plotting import plot_micro_f1_at_k

In [5]:
MODELS_ROOT = os.path.abspath("../../../models/ranking/movielens-ovr-linear-svc-calibrated/")
INTERIM_DATA_ROOT = os.path.abspath("../../../data/interim/movielens-ml20m-imdb/")
PATH_TO_PROCESSED_FILE = os.path.abspath('../../../data/processed/movielens-20m-imdb-tags-and-synopses-2017-12-20.csv')

# CONFIGS
SEED= 42

MAX_NB_WORDS = 1000

In [6]:
np.random.seed(SEED)

In [7]:
docs_df = load_df_or_get_from_cache(PATH_TO_PROCESSED_FILE,INTERIM_DATA_ROOT)

In [8]:
labels = docs_df["tags"].map(lambda tagstring: tagstring.split(","))

In [9]:
mlb = MultiLabelBinarizer()

binary_labels = mlb.fit_transform(labels)

print("total number of unique tags: {} ".format(len(mlb.classes_)))

data = docs_df['synopsis'].values
indices = np.arange(len(data))

np.random.shuffle(indices)

data = [data[i] for i in indices]
targets = binary_labels[indices]
num_validation_samples = int(0.15 * len(data))

X_train = data[:-num_validation_samples]
Y_train = targets[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = targets[-num_validation_samples:]

print('total number of train documents: {}'.format(len(X_train)))
print('total number of validation documents: {}'.format(len(X_val)))

total number of unique tags: 2138 
total number of train documents: 5704
total number of validation documents: 1006


In [16]:
# only fit on the train data because we're using the IDF scores to weigh the embeddings
vect = TfidfVectorizer(max_features=MAX_NB_WORDS)
vect.fit(X_train)

feature_names = vect.get_feature_names()
idf = vect.idf_
idf_index = dict(zip(vect.get_feature_names(), idf))

## tokenize

In [32]:
# analyzer = preprocess + tokenize
tokenize_func = vect.build_analyzer()

In [33]:
def tokenize(string):
    return tokenize_func(string)

In [37]:
X_train_tok = Parallel(n_jobs=2)(delayed(tokenize)(doc) for doc in X_train)
X_val_tok = Parallel(n_jobs=2)(delayed(tokenize)(doc) for doc in X_val)

## transform into embeddings

In [20]:
embeddings_index = read_glove_wiki_weighted(
    d=100,
    weight_index=idf_index)

overall, 1000 out of 1000 embeddings were weighted. Total available embeddings: 400000


In [41]:
def build_bag_of_weighted_embeddings(document_tokens):
    out = [embeddings_index[token] for token in document_tokens if token in embeddings_index.keys()]
    
    return np.mean(np.array(out),axis=0)    

In [48]:
X_train_boe = Parallel(n_jobs=2)(delayed(build_bag_of_weighted_embeddings)(tokenized_doc) for tokenized_doc in X_train_tok)
X_train_boe = np.array(X_train_boe)

X_val_boe = Parallel(n_jobs=2)(delayed(build_bag_of_weighted_embeddings)(tokenized_doc) for tokenized_doc in X_val_tok)
X_val_boe = np.array(X_val_boe)

In [49]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

pipeline = Pipeline([
    ('clf', OneVsRestClassifier(CalibratedClassifierCV(SVC(),cv=2),n_jobs=-1)),
])

In [51]:
parameters = [
    { 
        "clf__estimator__base_estimator__kernel": ['rbf','poly','linear'],
        "clf__estimator__base_estimator__degree": [2,3,4,5]
    }
]

In [50]:
pipeline.get_params()

{'clf': OneVsRestClassifier(estimator=CalibratedClassifierCV(base_estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
             cv=2, method='sigmoid'),
           n_jobs=-1),
 'clf__estimator': CalibratedClassifierCV(base_estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
             cv=2, method='sigmoid'),
 'clf__estimator__base_estimator': SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 'clf__estimator__base_estimator__C': 1.0,
 'c

In [ ]:
%%time

for g in tqdm(ParameterGrid(parameters)):
    print(g)
    
    pipeline.set_params(**g)
    
    pipeline.fit(X_train_boe,Y_train)
    
    Y_pred_train = pipeline.predict_proba(X_train_boe)    
    Y_pred_val = pipeline.predict_proba(X_val_boe)
    
    train_scores = [ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True) for k in ks]
    validation_scores = [ranking.micro_f1_at_k(Y_val,Y_pred_val,k=k,normalize=True) for k in ks]
    
    print(train_scores)
    print(validation_scores)
    

  0%|          | 0/12 [00:00<?, ?it/s]

{'clf__estimator__base_estimator__kernel': 'rbf', 'clf__estimator__base_estimator__degree': 2}


In [ ]:
ks = [1,2,3,4,5,6,7,8,9,10]

train_scores = [ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True) for k in ks]
validation_scores = [ranking.micro_f1_at_k(Y_val,Y_pred_val,k=k,normalize=True) for k in ks]

In [ ]:
plt.clf()
img = plt.gcf()
ax = plt.gca()
plot_micro_f1_at_k(validation_scores,ax,train_scores)
plt.gcf().set_size_inches(7,5)
plt.show()